In [50]:
from datascience import *
from scipy import stats
from scipy import special
import numpy as np 
import pandas as pd
import math
import random
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth 
import time 
from pprint import pprint

#variables necessary for spotipy
cid = '03b0d5c7f4184ca681429c98886ff58e'
secret = 'e46494252d5147d9a633698d8276ce7f'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### User input

In [51]:
new_playlist_name = 'Spotify Recommended Playlist #1'

playlist_1_link = 'https://open.spotify.com/playlist/4dBlUz2BDXmzcbvdnfp0bs'
playlist_2_link = 'https://open.spotify.com/playlist/0RNqeh2UurEc8NqTJxvHP6'

# % of songs from the added playlists to be outputted 
# (value between 0 to 1)
total_output = 0.8 

# % of output returned that should be new songs not in either playlist
# (value between 0 and 0.5)
percent_new = 0.5 

In [52]:
playlist_1 = sp.playlist_items(playlist_1_link)
playlist_2 = sp.playlist_items(playlist_2_link)

### Basic summary statistics

In [53]:
playlist_1_length = len(playlist_1['items'])
playlist_2_length = len(playlist_2['items'])
total_length = playlist_1_length + playlist_2_length 
shorter_playlist = 'null'

if playlist_1_length >= playlist_2_length:
    shorter_playlist = 'Playlist 2'
else:
    shorter_playlist = 'Playlist 1'

shortest_length = 0

if shorter_playlist == 'Playlist 1':
    shortest_length = playlist_1_length + 0
else:
    shortest_length = playlist_2_length + 0

In [54]:
percentage_output = total_output * 100
num_songs_output = int(total_output * shortest_length)
percentage_new = percent_new * 100
num_new_songs = int(num_songs_output * percent_new)
num_og_songs = num_songs_output - num_new_songs 

In [55]:
print(f'The number of tracks in both playlists combined is {total_length}.')
print(f'The length of the shortest playlist, {shorter_playlist}, is {shortest_length}.')
print(f'The number of tracks to be outputted is {percentage_output}% of the shorter playlist, which is {num_songs_output} tracks.')
print(f'Of the songs outputted, {percentage_new}%, or {num_new_songs} tracks, will be newly recommended songs, and {num_og_songs} will be from the \ninputted playlists.')

The number of tracks in both playlists combined is 130.
The length of the shortest playlist, Playlist 2, is 30.
The number of tracks to be outputted is 80.0% of the shorter playlist, which is 24 tracks.
Of the songs outputted, 50.0%, or 12 tracks, will be newly recommended songs, and 12 will be from the 
inputted playlists.


### Combine playlists from user input

In [56]:
# Creating lists of song URIs for each playlist
playlist_1_ids = [] 
playlist_2_ids = []  

for elem in np.arange(playlist_1_length):
    track_id = playlist_1['items'][elem]['track']['id']
    if track_id not in playlist_1_ids and track_id not in playlist_2_ids:
        playlist_1_ids.append(track_id)

for elem in np.arange(playlist_2_length):
    track_id = playlist_2['items'][elem]['track']['id']
    if track_id not in playlist_1_ids and track_id not in playlist_2_ids:
        playlist_2_ids.append(track_id)

playlist_1_length = len(playlist_1_ids)
playlist_2_length = len(playlist_2_ids)

### Extract and clean data

In [57]:
column_labels = list(sp.audio_features(playlist_1_ids)[0].keys())

# Playlist 1
playlist_1_data = {}

for attr in column_labels:
    playlist_1_data[attr] = []

playlist_1_names_list = [] # Scrape the names of all the tracks

for elem in playlist_1_ids:
    track_name = sp.tracks([elem])['tracks'][0]['name']
    playlist_1_names_list.append(track_name)
playlist_1_data['name'] = playlist_1_names_list 

ones_list = [] 
for elem in np.arange(playlist_1_length):
    ones_list.append(1)
playlist_1_data['og playlist'] = ones_list 
for song in playlist_1_ids:
    all_features = sp.audio_features(song)[0]
    for attr in column_labels: 
        value_of_interest = all_features[attr]
        (playlist_1_data[attr]).append(value_of_interest)

# Playlist 2
playlist_2_data = {}

for attr in column_labels:
    playlist_2_data[attr] = []

playlist_2_names_list = [] 

for elem in playlist_2_ids:
    track_name = sp.tracks([elem])['tracks'][0]['name']
    playlist_2_names_list.append(track_name)
playlist_2_data['name'] = playlist_2_names_list 
twos_list = [] 

for elem in np.arange(playlist_2_length):
    twos_list.append(2)
playlist_2_data['og playlist'] = twos_list 

for song in playlist_2_ids:
    all_features = sp.audio_features(song)[0]
    for attr in column_labels:
        value_of_interest = all_features[attr]
        (playlist_2_data[attr]).append(value_of_interest)

In [80]:
playlist_1_table = pd.DataFrame(playlist_1_data)
playlist_2_table = pd.DataFrame(playlist_2_data)
frames = [playlist_1_table, playlist_2_table]
combined_table = (pd.concat(frames)).set_index('name')
combined_table 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,og playlist
name,,,,,,,,,,,,,,,,,,,
Desafinado,0.501,0.186,10,-17.522,1,0.0544,0.9120,0.002640,0.1160,0.421,144.081,audio_features,2tcfbkABIwzpepPyaFsYF4,spotify:track:2tcfbkABIwzpepPyaFsYF4,https://api.spotify.com/v1/tracks/2tcfbkABIwzp...,https://api.spotify.com/v1/audio-analysis/2tcf...,248013,4,1
Só Danço Samba,0.578,0.364,2,-15.162,1,0.0441,0.6830,0.053900,0.1050,0.723,78.073,audio_features,5je0CDv1Z1hT82Nek1Fjhl,spotify:track:5je0CDv1Z1hT82Nek1Fjhl,https://api.spotify.com/v1/tracks/5je0CDv1Z1hT...,https://api.spotify.com/v1/audio-analysis/5je0...,216227,4,1
These Foolish Things (Remind Me of You) - Remastered 2020,0.443,0.283,1,-12.977,1,0.0397,0.9150,0.899000,0.1550,0.354,171.005,audio_features,1c5igSyxa7cr4goWuK2ulv,spotify:track:1c5igSyxa7cr4goWuK2ulv,https://api.spotify.com/v1/tracks/1c5igSyxa7cr...,https://api.spotify.com/v1/audio-analysis/1c5i...,423620,3,1
Wave,0.552,0.314,7,-20.681,1,0.0337,0.8310,0.831000,0.0861,0.873,144.921,audio_features,2hXBS8q9rGMovfG1S8FB4F,spotify:track:2hXBS8q9rGMovfG1S8FB4F,https://api.spotify.com/v1/tracks/2hXBS8q9rGMo...,https://api.spotify.com/v1/audio-analysis/2hXB...,176907,4,1
Choro,0.724,0.289,10,-21.862,0,0.0622,0.6200,0.819000,0.1160,0.852,139.786,audio_features,7AD0HyWHuDjcS74zGAzOhO,spotify:track:7AD0HyWHuDjcS74zGAzOhO,https://api.spotify.com/v1/tracks/7AD0HyWHuDjc...,https://api.spotify.com/v1/audio-analysis/7AD0...,128280,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bouncing with Bud,0.436,0.305,7,-19.361,0,0.0426,0.7890,0.940000,0.1070,0.783,126.864,audio_features,5gNAedRB0q6RR466babDiq,spotify:track:5gNAedRB0q6RR466babDiq,https://api.spotify.com/v1/tracks/5gNAedRB0q6R...,https://api.spotify.com/v1/audio-analysis/5gNA...,266731,4,2
Love On A Two Way Street,0.589,0.461,2,-12.676,1,0.0345,0.8350,0.765000,0.1250,0.497,77.221,audio_features,2NF1nbpz15lOjKKLm9jMb0,spotify:track:2NF1nbpz15lOjKKLm9jMb0,https://api.spotify.com/v1/tracks/2NF1nbpz15lO...,https://api.spotify.com/v1/audio-analysis/2NF1...,287360,4,2
They Tell Me It's Summer,0.623,0.177,5,-19.030,1,0.0278,0.6620,0.000028,0.1130,0.495,101.187,audio_features,6FgkNsnyK4JujUKkvgLGtL,spotify:track:6FgkNsnyK4JujUKkvgLGtL,https://api.spotify.com/v1/tracks/6FgkNsnyK4Ju...,https://api.spotify.com/v1/audio-analysis/6Fgk...,149493,4,2


In [59]:
combined_tracks_id_list = list(combined_table.index)
combined_tracks_name_list = combined_table['name'].tolist()
id_to_name = {}

for elem in combined_tracks_id_list:
    id_to_name[elem] = combined_tracks_name_list[combined_tracks_id_list.index(elem)]

### Weights

In [60]:
weights_dictionary = {
    'genre_weight':17,   
    'artist_weight':12,  
    'mode_weight':8,   
    'valence_weight':15,
    'tempo_weight':6,   
    'danceability_weight':13,
    'energy_weight':11,
    'acousticness_weight':5,
    'instrumentalness_weight':3,
    'loudness_weight':0,
    'liveness_weight':5,
    'speechiness_weight':5
                        }

In [61]:
weights_total = sum(list(weights_dictionary.values()))

assert sum(list(weights_dictionary.values())) == 100, f'Weights do not add up to 100, adds to {weights_total}'

focus_attributes = ['danceability', 'valence', 'energy', 'liveness', 'speechiness'] 
print(f'Current attributes considered are: {focus_attributes}')

Current attributes considered are: ['danceability', 'valence', 'energy', 'liveness', 'speechiness']


### Create a function that calculates similarity between two songs based on weights

In [62]:
tbl = combined_table 

def similarity(song_a, song_b, attributes):
    """
    Takes in two songs and calculates a similarity score by multiplying percent similarity
    in focus attributes by the weights of those respective attributes and summing together
    """
    similarity_sum = 0
    for elem in attributes:
        a_value = tbl.loc[f'{song_a}'][f'{elem}']
        b_value = tbl.loc[f'{song_b}'][f'{elem}']
        att_weight = weights_dictionary[f'{elem}_weight']
        value_diff = abs(a_value - b_value)
        percent_similar = value_diff / ((a_value + b_value) / 2) # Comparing difference value to the mean of the two numbers
        weighted_similarity = percent_similar * att_weight
        similarity_sum = similarity_sum + weighted_similarity
    return similarity_sum 

In [63]:
song_matrix = pd.DataFrame(0, index=combined_tracks_id_list , columns=combined_tracks_id_list, dtype=float)

# Set each value that is a song to itself to 1
for elem in combined_tracks_id_list:
    song_matrix.at[f'{elem}', f'{elem}'] = 1

for song1 in combined_tracks_id_list:
    focus_song = song1
    for song2 in combined_tracks_id_list:
        if song1 != song2:
            song_matrix.at[f'{song1}', f'{song2}'] = similarity(song1, song2, focus_attributes)

In [64]:
# Song matrix that averages scores for each song
average_similarity_scores = []

for elem in song_matrix.index.values.tolist():
    score_list = list(song_matrix.get(f'{elem}').values)
    average_score = sum(score_list) / len(score_list)
    average_similarity_scores.append(average_score)
song_matrix['average'] = average_similarity_scores
song_matrix['average']

2tcfbkABIwzpepPyaFsYF4    23.288700
5je0CDv1Z1hT82Nek1Fjhl    25.933243
1c5igSyxa7cr4goWuK2ulv    23.373777
2hXBS8q9rGMovfG1S8FB4F    27.893151
7AD0HyWHuDjcS74zGAzOhO    29.726511
                            ...    
5gNAedRB0q6RR466babDiq    26.096937
2NF1nbpz15lOjKKLm9jMb0    25.032874
6FgkNsnyK4JujUKkvgLGtL    24.877243
6I7U7kjLwwS3UYbGCNLcx0    26.195249
6FpDOUWn8aIjL64PyIGjvu    32.111340
Name: average, Length: 126, dtype: float64

In [65]:
def typeConverter(x): 
    """
    In the case where the matrix is an array (may happen when songs are repeated in playlist), 
    this will remove one of them.
    """
    y = []
    for val in x: 
        if type(val) == np.ndarray:
            y.append(val[0])
        else:
            y.append(val)
    return y

index = song_matrix.index
newAverage = typeConverter(average_similarity_scores)
zipValues = list(zip(index, newAverage))
zipValues.sort(key=lambda x: x[1], reverse=True)
     
def make_generator(n):
    while len(n) >= 0:
        yield n[0][0]
        n = n[1:]

copy_zipValues = list(zipValues)
sorted_generator = make_generator(copy_zipValues)

In [66]:
og_similar_songs = []

x = num_og_songs + 0
playlist_1_counter = 0
playlist_2_counter = 0
target_playlist_1_max = int(0.7 * num_og_songs)
target_playlist_2_max = int(0.7 * num_og_songs)
while x != 0 and (len(og_similar_songs) != num_og_songs):
    song_id = next(sorted_generator)
    song_title = id_to_name[song_id]
    if song_id in playlist_1_ids and playlist_1_counter <= target_playlist_1_max:
        playlist_1_counter += 1
        x -= 1
        og_similar_songs.append(song_title)
    if song_id in playlist_2_ids and playlist_2_counter <= target_playlist_2_max:
        playlist_2_counter += 1
        x -= 1
        og_similar_songs.append(song_title)

print(f'The {num_og_songs} songs from both playlists to be returned are:')
pprint(og_similar_songs, indent=2)

The 12 songs from both playlists to be returned are:
[ 'aisatsana [102]',
  'A Flower is not a Flower',
  'All Across The City',
  'Like Someone In Love',
  'That Secret Place',
  "I'm All Smiles",
  'Peace Piece',
  'Why Did I Choose You?',
  "While We're Young",
  'Contact',
  'Juicy Fruit',
  'Bite the Bit']


### Group songs together by their main attributes

In [67]:
original_songs_dictionary = id_to_name 
og_song_ids = list(original_songs_dictionary.keys())
recommendation_attributes = ['valence', 'tempo', 'energy']

In [68]:
def sort_tuple_list(tup):
    return(sorted(tup, key = lambda x: x[1]))

In [69]:
# Valence
attribute = 'valence'

valence_list = list(combined_table.loc[:, f'{attribute}'])
valence_table = combined_table.loc[:, ['name', f'{attribute}']]
valence_tuple_list = []
for item in np.arange(len(list(valence_list))):
    track_id = (valence_table.index.values.tolist())[item]
    valence_value = list(valence_table[f'{attribute}'])[item]
    valence_tuple = (track_id, valence_value)
    valence_tuple_list.append(valence_tuple)
ordered_valence_tuples = sort_tuple_list(valence_tuple_list)

# Group elements
ordered_valences = []
for elem in ordered_valence_tuples:
    ordered_valences.append(elem[1])

diff = [y - x for x, y in zip(*[iter(ordered_valences)] * 2)]
avg = sum(diff) / len(diff) 

valence_groups = [[ordered_valences[0]]]
for x in ordered_valences[1:]:
    if x - valence_groups[-1][0] < avg:
        valence_groups[-1].append(x)
    else:
        valence_groups.append([x])

for elem in valence_groups:
    for item in elem:
        if item in ordered_valences:
            origin = elem.index(item)
            position = ordered_valences.index(item)
            track_id = ordered_valence_tuples[position][0]
            elem.insert(origin, track_id)
            elem.remove(item)

#print('Valence groups:')
#print(valence_groups)

In [70]:
# Danceablility
attribute = 'danceability'

danceability_list = list(combined_table.loc[:, f'{attribute}'])
danceability_table = combined_table.loc[:, ['name', f'{attribute}']]
danceability_tuple_list = []
for item in np.arange(len(list(danceability_list))):
    track_id = (danceability_table.index.values.tolist())[item]
    danceability_value = list(danceability_table[f'{attribute}'])[item]
    danceability_tuple = (track_id, danceability_value)
    danceability_tuple_list.append(danceability_tuple)
ordered_danceability_tuples = sort_tuple_list(danceability_tuple_list)

# Group elements
ordered_danceabilitys = []
for elem in ordered_danceability_tuples:
    ordered_danceabilitys.append(elem[1])

diff = [y - x for x, y in zip(*[iter(ordered_danceabilitys)] * 2)]
avg = sum(diff) / len(diff) 

danceability_groups = [[ordered_danceabilitys[0]]]
for x in ordered_danceabilitys[1:]:
    if x - danceability_groups[-1][0] < avg:
        danceability_groups[-1].append(x)
    else:
        danceability_groups.append([x])

for elem in danceability_groups:
    for item in elem:
        if item in ordered_danceabilitys:
            origin = elem.index(item)
            position = ordered_danceabilitys.index(item)
            track_id = ordered_danceability_tuples[position][0]
            elem.insert(origin, track_id)
            elem.remove(item)

#print('Danceability groups:')
#print(danceability_groups)

In [71]:
# Energy
attribute = 'energy'

energy_list = list(combined_table.loc[:, f'{attribute}'])
energy_table = combined_table.loc[:, ['name', f'{attribute}']]
energy_tuple_list = []
for item in np.arange(len(list(energy_list))):
    track_id = (energy_table.index.values.tolist())[item]
    energy_value = list(energy_table[f'{attribute}'])[item]
    energy_tuple = (track_id, energy_value)
    energy_tuple_list.append(energy_tuple)
ordered_energy_tuples = sort_tuple_list(energy_tuple_list)

# Group elements

ordered_energys = []
for elem in ordered_energy_tuples:
    ordered_energys.append(elem[1])

diff = [y - x for x, y in zip(*[iter(ordered_energys)] * 2)]
avg = sum(diff) / len(diff) 

#actual grouping
energy_groups = [[ordered_energys[0]]]
for x in ordered_energys[1:]:
    if x - energy_groups[-1][0] < avg:
        energy_groups[-1].append(x)
    else:
        energy_groups.append([x])

for elem in energy_groups:
    for item in elem:
        if item in ordered_energys:
            origin = elem.index(item)
            position = ordered_energys.index(item)
            track_id = ordered_energy_tuples[position][0]
            elem.insert(origin, track_id)
            elem.remove(item)
            
#print('Energy groups:')
#print(energy_groups)

In [72]:
# Aggregate elements into one list and sort by length 

aggregated_groups = []
for elem in valence_groups:
    aggregated_groups.append(elem)
for elem in danceability_groups:
    aggregated_groups.append(elem)
for elem in energy_groups:
    aggregated_groups.append(elem)

ordered_aggregated_groups = sorted(aggregated_groups, key=lambda x: len(x), reverse=True)  #this sorts in descending order the length of groups

interest_groups = []
counter = num_new_songs

while counter != 0:
    interest_groups.append(ordered_aggregated_groups[num_new_songs - counter])
    counter -= 1

### Insert into Spotipy recommendation function

In [73]:
final_song_ids = []
new_songs_ids = []

for elem in interest_groups:
    if len(elem) > 5: 
        edited = []
        limit = 5
        while limit != 0:
            edited.append(elem[5 - limit])
            limit -= 1
        elem = edited
    song_added = False
    raw_rec = sp.recommendations(seed_tracks=elem, limit=10)
    possible_tracks = raw_rec['tracks']

    list_of_possible = []
    for item in possible_tracks:
        item_id = item['id']
        list_of_possible.append(item_id)
    while song_added == False:
        random_track = random.choice(list_of_possible)
        if random_track not in final_song_ids:
            if random_track not in og_song_ids:
                song_added = True
                new_songs_ids.append(random_track)

In [74]:
new_song_dict = {}
for elem in new_songs_ids:
    name = sp.track(elem)['name']
    new_song_dict[elem] = name

new_songs_names = []
for elem in new_songs_ids:
    name = new_song_dict[elem]
    new_songs_names.append(name)

new_songs_table = Table().with_column("Name", new_songs_names)
new_songs_table

Name
You've Changed - Remastered 2015
Maybe September - Remastered 2003
Break of Dawn
You Don't Know What Love Is
Alice In Wonderland - Take 2 / Live At The Village Vangu ...
Nostalgic Montage
A Remark You Made
Equanimity
Lonely Woman - Remastered 1999/Rudy Van Gelder Edition
For All We Know


In [75]:
final_songs = og_similar_songs + new_songs_names
og_keys = list(original_songs_dictionary.keys())
og_values = list(original_songs_dictionary.values())
new_keys = list(new_song_dict.keys())
new_values = list(new_song_dict.values())
concat_keys = og_keys + new_keys 
concat_values = og_values + new_values
reference_dictionary = {}
reference_length = len(concat_keys)
for elem in np.arange(reference_length):
    reference_dictionary[concat_keys[elem]] = concat_values[elem]

final_songs_dictionary = {}

def get_key(val):
    for key, value in reference_dictionary.items():
        if val == value:
            return key
for elem in final_songs:
    final_songs_dictionary[elem] = get_key(elem)

### Final playlist

In [77]:
print(f"The final playlist, '{new_playlist_name}', of {num_songs_output} tracks is ")
pprint(final_songs, indent = 2)

The final playlist, 'Spotify Recommended Playlist #1', of 24 tracks is 
[ 'aisatsana [102]',
  'A Flower is not a Flower',
  'All Across The City',
  'Like Someone In Love',
  'That Secret Place',
  "I'm All Smiles",
  'Peace Piece',
  'Why Did I Choose You?',
  "While We're Young",
  'Contact',
  'Juicy Fruit',
  'Bite the Bit',
  "You've Changed - Remastered 2015",
  'Maybe September - Remastered 2003',
  'Break of Dawn',
  "You Don't Know What Love Is",
  'Alice In Wonderland - Take 2 / Live At The Village Vanguard, NYC; 6/25/1961',
  'Nostalgic Montage',
  'A Remark You Made',
  'Equanimity',
  'Lonely Woman - Remastered 1999/Rudy Van Gelder Edition',
  'For All We Know',
  'Shinsen',
  'Song For Fraser']
